In [1]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("C:\\Users\\bordi\\OneDrive\\Desktop\\epds\\2021-2022\\resources\\artchives.nq", format='nquads')

In [2]:
colls_and_people = []

# get all the collections
for coll,isA,coll_class in g.triples(( None, RDF.type , wd.Q9388534)): 
    # create a list for each collection
    coll_list = [] 
    # get all the periods for that collection
    for this_coll, hasSubj, people in g.triples(( coll, art.hasSubjectPeople, None)): 
        # get the labels of the periods
        for this_people, hasLabel, people_label in g.triples((people , RDFS.label, None )):
            # get only the last label
            people_label = str(people_label).strip() 
        # append periods to the related collection
        coll_list.append(people_label)
    # append collections to the initial list
    colls_and_people.append(coll_list)
    
print(colls_and_people)

[['Federico Zeri', 'John Pope-Hennessy', 'David Freedberg', 'Andrea de Marchi'], ['Everett Fahy', 'Roberto Longhi', 'Bernard Berenson', 'Vittorio Cini', 'Alessandro Contini Bonacossi', 'galleria Sangiorgi', 'Umberto Gnoli', 'Evelyn Sandberg Vavalà', 'Antonio Muñoz', 'Guglielmo Matthiae'], ['Massimo Ferretti', 'Andrea Emiliani', 'Anna Ottani Cavina', 'Andrea Bacchi'], ['R.F. Arragon', 'H. Berlin;', 'R. Berliner;', 'H.C. Bernard;', 'F.A. Bernett;', 'K.G. Bernheimer; J.A. Bernstein;', 'A. Bartolini Salimbeni;', 'E.M. Becker;', 'B. Bearzi', 'M. Barsanti;', 'H. Beckenbach; E.M. Becker;', 'N. Bemporad;', 'Bemporad-Marzocco; E. Bendheim', 'S. Bene', 'O. Benesch', 'J. Bennewitz', 'M. Berchtold', 'E. Berg', 'K. Berg', 'K. Berger', 'P. Berger', 'A. Bergsträsser', 'C. Bertelli', 'S. Bertelli', 'F. Bertelotti', 'A. Bertini', 'L. Berti', 'R. Bertos', 'A. Bettagno', 'G. Betz', 'P. Biagini', 'L. Bianchi', 'G.P. Biasin', 'J. Biakostocki', 'Bibliotheka Hertziana', 'F.M. Biebel', 'E. Bielefeld', 'H.N. B

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import pandas as pd

dataset = colls_and_people

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.08,(Alessandro Contini Bonacossi)
1,0.08,(Bernard Berenson)
2,0.08,(Ernst Kitzinger)
3,0.12,(Ernst Steinmann)
4,0.20,(Erwin Panofsky)
5,0.12,(Federico Zeri)
6,0.08,(Kunsthistorisches Institut in Florenz)
7,0.08,(Leo Bruhns)
8,0.08,(Otto Lehmann-Brockhaus)
9,0.08,(Pompeo Molmenti)


In [5]:
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Alessandro Contini Bonacossi),(Bernard Berenson),0.08,0.08,0.08,1.0,12.500000,0.0736,inf
1,(Bernard Berenson),(Alessandro Contini Bonacossi),0.08,0.08,0.08,1.0,12.500000,0.0736,inf
2,(Ernst Kitzinger),(Erwin Panofsky),0.08,0.20,0.08,1.0,5.000000,0.0640,inf
3,(Wolfgang Lotz),(Ernst Steinmann),0.08,0.12,0.08,1.0,8.333333,0.0704,inf
4,(Willibald Sauerländer),(Erwin Panofsky),0.08,0.20,0.08,1.0,5.000000,0.0640,inf
5,(Richard Krautheimer),(Otto Lehmann-Brockhaus),0.08,0.08,0.08,1.0,12.500000,0.0736,inf
6,(Otto Lehmann-Brockhaus),(Richard Krautheimer),0.08,0.08,0.08,1.0,12.500000,0.0736,inf
